# DRL portfolio allocation example

- Based on FinRL documentation
- cleaned up

# Preface

Needed Python packages

- finrl
- gymnasium
- stable-baselines3[extra]
- exchange-calendars
- stockstats
- wrds

# Parameters

- In this section configure the parameters for this backtest

In [5]:
import os
from pathlib import Path

# config = dynamic_import("config", CURRENT_PATH + "/config.py")
# config_tickers = dynamic_import("config_tickers", CURRENT_PATH + "/config_tickers.py")

PREPARED_DATA_FILE = Path("/tmp/finrl-demo.parquet")
TRAINED_MODEL_FILE = Path("/tmp/yahoo-dow-jones-index-model.pkl")

# Asset universe we use
DOW_30_TICKER = [
    "AXP",
    "AMGN",
    "AAPL",
    "BA",
    "CAT",
    "CSCO",
    "CVX",
    "GS",
    "HD",
    "HON",
    "IBM",
    "INTC",
    "JNJ",
    "KO",
    "JPM",
    "MCD",
    "MMM",
    "MRK",
    "MSFT",
    "NKE",
    "PG",
    "TRV",
    "UNH",
    "CRM",
    "VZ",
    "V",
    "WBA",
    "WMT",
    "DIS",
    "DOW",
]

INDICATORS = [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
]


START_DATE = "2010-01-01"

TRAIN_START_DATE = "2014-01-01"
TRAIN_END_DATE = "2020-07-31"

TEST_START_DATE = "2020-08-01"
TEST_END_DATE = "2021-10-01"

TRADE_START_DATE = "2021-11-01"
TRADE_END_DATE = "2021-12-01"



''

# Download and setup trading universe

In [6]:
import os

import pandas as pd
from stable_baselines3 import A2C

from agents.stablebaselines3_models import DRLAgent
from meta import config
from meta import config_tickers
from meta.data_processor import DataProcessor
from meta.env_portfolio_allocation.env_portfolio_yahoofinance import StockPortfolioEnv



def setup_data(
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    ticker_list=DOW_30_TICKER,
    time_interval="1D",
    data_source="yahoofinance",
    technical_indicator_list=INDICATORS,
    if_vix=True,
    hmax=100,
    initial_amount=1000000,
    transaction_cost_pct=0.001,
    reward_scaling=1e-4,
    use_cached_model=False,
) -> pd.DataFrame:
    
    # Cache prepared data locally
    if PREPARED_DATA_FILE.exists():
        print(f"Using cached data from {PREPARED_DATA_FILE}")
        df = pd.read_parquet(PREPARED_DATA_FILE)
        return df

    # download data
    dp = DataProcessor(
        data_source=data_source,
        start_date=start_date,
        end_date=end_date,
        time_interval=time_interval,
    )

    price_array, tech_array, turbulence_array = dp.run(
        ticker_list,
        technical_indicator_list,
        if_vix=if_vix,
        cache=True,
        select_stockstats_talib=0,
    )

    # add covariance matrix as states
    df = dp.dataframe
    df = df.sort_values(["time", "tic"], ignore_index=True)
    df.index = df.time.factorize()[0]

    df["pct_change"] = df.groupby("tic").close.pct_change()

    cov_list = []
    # look back is one year
    lookback = 252
    for i in range(lookback, len(df.index.unique())):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(
            index="time", columns="tic", values="close"
        )
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values
        cov_list.append(covs)
    # df["mean_pct_change_lookback"] = df.rolling(lookback)["pct_change"].mean()
    # df["ewm_returns"] = df["pct_change"].ewm(span=50).mean()
    df_cov = pd.DataFrame({"time": df.time.unique()[lookback:], "cov_list": cov_list})
    df = df.merge(df_cov, on="time")
    df = df.sort_values(["time", "tic"]).reset_index(drop=True)

    print("Saving prepared data to", PREPARED_DATA_FILE)
    print("Columns are", df.columns)
    df.write.parquet(PREPARED_DATA_FILE)

    return df

    

df = setup_data()

print("Prepared data is:")
display(df)


ModuleNotFoundError: No module named 'agents'

# Train